In [89]:
from math import log, pow, exp, sqrt, fabs, e
from scipy.stats import norm
import pandas as pd
from datetime import date
import numpy as np
from py_vollib.black_scholes.greeks.analytical import delta, gamma, vega, theta

In [103]:
class Option:
    """
    Calculate the Greeks letter for Option 
    """
    def __init__(self, right, s, k, eval_date, exp_date, price = None, rf = 0.0, vol = 0.3,
                 div = 0):
        self.k = float(k)
        self.s = float(s)
        self.rf = float(rf)
        self.vol = float(vol)
        self.eval_date = eval_date
        self.exp_date = exp_date
        self.t = self.calculate_t()
        self.price = price
        self.right = right   ## 'C' or 'P'
        self.div = div
        self.d1 = (log(self.s/self.k) + (self.rf + self.div + pow(self.vol, 2)/2 ) * self.t) / (self.vol * sqrt(self.t))
        self.d2 = self.d1 - self.vol * sqrt(self.t)

    def calculate_t(self):
        if isinstance(self.eval_date, str):
            if '/' in self.eval_date:
                (day, month, year) = self.eval_date.split('/')
            else:
                (day, month, year) = self.eval_date[6:8], self.eval_date[4:6], self.eval_date[0:4]
            d0 = date(int(year), int(month), int(day))
        elif type(self.eval_date)==float or type(self.eval_date)==long or type(self.eval_date)==np.float64:
            (day, month, year) = (str(self.eval_date)[6:8], str(self.eval_date)[4:6], str(self.eval_date)[0:4])
            d0 = date(int(year), int(month), int(day))
        else:
            d0 = self.eval_date 

        if isinstance(self.exp_date, str):
            if '/' in self.exp_date:
                (day, month, year) = self.exp_date.split('/')
            else:
                (day, month, year) = self.exp_date[6:8], self.exp_date[4:6], self.exp_date[0:4]
            d1 = date(int(year), int(month), int(day))
        elif type(self.exp_date)==float or type(self.exp_date)==long or type(self.exp_date)==np.float64:
            (day, month, year) = (str(self.exp_date)[6:8], str(self.exp_date)[4:6], str(self.exp_date)[0:4])
            d1 = date(int(year), int(month), int(day))
        else:
            d1 = self.exp_date

        t = np.busday_count(d0, d1)/252
        if t < 0:
            print("The input dates error!")
        return t
    
    def get_price(self):
        d1 = self.d1
        d2 = self.d2
        if self.right == 'C':
            self.calc_price = ( norm.cdf(d1) * self.s * exp(-self.div*self.t) - norm.cdf(d2) * self.k * exp( -self.rf * self.t ) )
        elif self.right == 'P':
            self.calc_price =  ( -norm.cdf(-d1) * self.s * exp(-self.div*self.t) + norm.cdf(-d2) * self.k * exp( -self.rf * self.t ))

        
    def get_delta(self):
        if self.right == 'C':
            self.delta = norm.cdf(self.d1)
        elif self.right == 'P':
            self.delta = -norm.cdf(-self.d1) 
 
 
    def get_theta(self):
        
        prob_density = norm.pdf(self.d1)
        if self.right == 'C':
            self.theta = (-self.vol * self.s * prob_density) / (2 * np.sqrt(self.t)) - self.rf * self.k * np.exp(-self.rf * self.t) * norm.cdf(self.d2)
        elif self.right == 'P':    
            self.theta = (-self.vol * self.s * prob_density) / (2 * np.sqrt(self.t)) + self.rf * self.k * np.exp(-self.rf * self.t) * norm.cdf(self.d2)
        self.theta = self.theta/365
        
    def get_gamma(self):
        self.gamma = e ** (-self.div * self.t) * norm.pdf(self.d1) / (self.s * self.vol* self.t ** 0.5)
        
 
    def get_vega(self):
        prob_density = norm.pdf(self.d1)
        print("self.d1:", self.d1)
        print("prob_density: ", prob_density)
        self.vega = 0.01 * self.s * prob_density * np.sqrt(self.t)
        

    def get_all(self):
        self.get_price()
        self.get_delta()
        self.get_theta()
        self.get_gamma()
        self.get_vega()
        
        return self.calc_price, self.delta, self.theta, self.gamma, self.vega
 

In [104]:
if __name__ ==  '__main__':
    s = 11368
    k = 11300
    exp_date = '20200219'
    eval_date = '20200130'
    rf = 0.0
    vol = 0.193986
    div = 0.0
    right = 'P'
    
    opt = Option(s=s, k=k, eval_date=eval_date, exp_date=exp_date, rf=rf, vol=vol, right=right,
                 div = div)
    price, delta, theta, gamma, vega = opt.get_all()
    

self.d1: 0.15407930548879584
prob_density:  0.39423474429638583


In [105]:
print("-------------- FIRST OPTION -------------------")
print("Price Put: " + str(price))  
print("Delta Put: " + str(delta)) 
print("Theta Put: " + str(theta)) 
print("Gamma Put:" + str(gamma))   
print("Vega Put:" + str(vega))

-------------- FIRST OPTION -------------------
Price Put: 174.50020052127456
Delta Put: -0.43877360262245524
Theta Put: -5.052691025605864
Gamma Put:0.0007584670108396254
Vega Put:10.563375274195849
